In [2]:
from src.utils.evaluation_utils import compute_f1, compute_exact

In [3]:
import pandas as pd
import numpy as np

In [7]:
from pathlib import Path

In [8]:
current_working_directory = Path.cwd().parent

In [4]:
data = pd.read_csv("models-predictions-fquad.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'models-predictions-fquad.csv'

In [5]:
data = data.dropna()

NameError: name 'data' is not defined

In [12]:
data["f1"] = np.vectorize(compute_f1)(data.prediction, data.goldlabel)
data["exact"] = np.vectorize(compute_exact)(data.prediction, data.goldlabel)

In [13]:
data.head()

,prediction,goldlabel,questions,f1,exact
0,la compression,la compression,question: Qu'est-ce qui réchauffe la substance...,1.0,1
1,en octobre 1854,octobre 1854,question: Quand est publiée la critique dans l...,0.8,0
2,à cause des limites imposées par la fonction d...,à cause des limites imposées par la fonction d...,question: Pourquoi le cheval ailé est-il coupé...,1.0,1
3,"l'association entre syndrome d'Asperger, maladie","l'association entre syndrome d'Asperger, maladie",question: Quelle confusion existe-t-il avec le...,1.0,1
4,Harrison Ford,Harrison Ford,question: Qui est finalement suggéré à Hampton...,1.0,1


In [14]:
data.f1.mean()

0.7499305076387526

In [17]:
data.loc[data.f1 ==0]

,prediction,goldlabel,questions,f1,exact
11,Théophraste,"du maître, mais aussi de l’intendant",question: Qui peut infliger la flagellation ? ...,0.0,0
19,Hayden,Ulysses Grant,question: Quel personnalité officialisa la pro...,0.0,0
25,sur le sol,au fond du puits,question: Où est-ce que gis Stephen ? </s>,0.0,0
53,William Grover-Williams-Caberto,Borzacchini,question: Qui devient le coéquipier de Nuvolar...,0.0,0
65,Cob normand,mareyeur Boulonnais,question: Quelle race autre que le cob est fav...,0.0,0
...,...,...,...,...,...
3156,Zèbre à Paris,sur différentes scènes nationales et internati...,question: Où Amalric assiste-t-il à des shows ...,0.0,0
3157,L'église est orientée,caractéristique préromane,question: Que caractérise l'emplacement de la ...,0.0,0
3171,imperceptibilité d'interpréter ses propres,Peu diplomates et peu persuas,question: Pourquoi les Asperger éprouvent des ...,0.0,0
3173,la plus complète définition qu'ait jamais donn...,mal social dans son ensemble,question: Que décrit le discours de Dickens d'...,0.0,0


In [21]:
data.questions = data.questions.str.replace("question:", "").str.replace("</s>", "")

In [68]:
data["question_type"] = data.questions.str.strip().str.split(" ").apply(lambda x: x[0].lower())
data["answer_length"] = data.goldlabel.str.split(" ").apply(lambda x: len(x))

In [40]:
question_prepositions = ["combien","quand", "qui", "quelle", "que", "pourquoi", "comment"]

In [69]:
data["question_type"].value_counts().loc[question_prepositions].sort_values(ascending=False)

qui         380
quelle      296
que         250
quand       186
comment     168
combien     154
pourquoi     63
Name: question_type, dtype: int64

In [70]:
question_type_count = data["question_type"].value_counts()

In [55]:
question_type_count.loc[question_type_count.values >=10].to_dict().keys()

dict_keys(['quel', 'qui', 'quelle', 'que', 'quand', 'comment', 'combien', 'de', 'où', 'dans', 'à', "qu'est-ce", 'a', 'quels', 'pourquoi', 'quelles', 'par', "qu'est", 'en', 'sur', 'pour', 'avec', 'selon', "d'où", 'depuis', 'lors'])

In [74]:
data['question_type'] = np.where(data['question_type'].isin(question_type_count.loc[question_type_count.values >=10].to_dict().keys()), data['question_type'], 'Other')

In [75]:
data_group_question = data.groupby('question_type').agg({'f1': 'mean', 'exact': 'mean', "prediction": "count", "answer_length": "mean"}).rename({"prediction": "count"}, axis=1)

In [77]:
data_group_question.sort_values(by=['answer_length', "f1"], ascending=True)

,f1,exact,count,answer_length
question_type,,,,
combien,0.801479,0.655844,154,2.090909
depuis,0.766162,0.416667,12,2.333333
qui,0.853524,0.794737,380,2.444737
quand,0.800437,0.596774,186,2.720430
lors,0.809048,0.500000,10,3.000000
quel,0.737137,0.588764,445,3.080899
quelle,0.769003,0.587838,296,3.101351
d'où,0.705263,0.684211,19,3.157895
a,0.774930,0.605634,71,3.183099


In [38]:
data.loc[data.question_type.str.lower() == "Quoi"]

,prediction,goldlabel,questions,f1,exact,question_type


In [17]:
experiments = pd.read_csv(current_working_directory.joinpath("experiments-resulsts.csv"),
                          usecols=["f1_score", "exact_matches", "Name", "Start Time"])

In [22]:
experiments.sort_values("Start Time", ascending=False).head(11)

,Start Time,Name,exact_matches,f1_score
0,2022-08-05 13:19:35,one-paragraph-piaf-multi-context,27.744833,45.604458
1,2022-08-05 13:04:28,one-pragraph-piaf-with-context,38.956974,55.874863
2,2022-08-05 12:44:44,one-paragraph-piaf-bm25,27.744833,45.604458
3,2022-08-05 12:35:48,one-prarragrah-fquad-bm25,54.673779,72.963928
4,2022-08-05 12:17:22,one-paragraph-fquad-valid,54.673779,72.963928
5,2022-08-05 12:09:23,one-pragraph-valid-fquad,NaN,NaN
6,2022-08-04 19:37:17,two-paragraph-piaf-base-questions,86.571053,91.426132
7,2022-08-04 19:17:24,two-paragraph-piaf-base-questions,86.571053,91.426132
8,2022-08-04 18:52:02,two-paragraph-piaf-additional-questions,35.489666,53.667488
9,2022-08-04 18:42:51,two-paragraph-fquad-with-multi-contexts-with-e...,57.685070,75.614594


In [ ]:
 

gcloud compute scp  --recurse data/corpus/french-qa/{fquad-with-multi-context-with-without-answer,piaf-additional-with-from-wikipedia-bm25} lafand-mt:/home/es.py/school_stuff/unsupervised-open-domain-french-question-answering/data/processed/french-qa